# Tree Based Methods

*First version on: 2020-11-11  
 Latest updates on: 2022-12-03*

## 1. Prediction/Decision Trees
What inspires the decision tree methodology?    

Predictive models like linear/polynomial regressions are **global models** which means the formula applies to the entire data space. When there are non-linear and complicated interatice effects among features, it is very hard to construct a **global** solution to do prediction.  

Some non-parametric smoothers try to fit the data **locally** and then integrate them. While for Prediction/Decision Trees, **sub-division** or **partition** is used to cut the data space into smaller manageable area (i.e. can be modeled using simple average or taking the majority of votes).

Note: This information is adapted from reference [1].

## 2. Regression Trees
### 2.1 What's the prediction mechanism for regression trees?
There are two steps for the training data:  
1. Divide the training predictor space (i.e. $X_1, X_2, \cdots, X_p$) into $J$ distinct and non-overlapping regions, $R_1, R_2, \cdots, R_J$.
2. For every observation that falls into the region $R_j$, we use the same value for its prediction. The prediction value is the mean of the response values of all the training observation in $R_j$.   

How do we construct the regions then?  
Let's define the objective function using residual sum of squares (i.e. *RSS*):
$$ RSS = \sum_{j=1}^{J}\sum_{i \in R_j} \left( y_i - \hat{y}_{R_j} \right)^{2}$$
where $\hat{y}_{R_j}$ is the mean response for the training observations within the $j^{th}$ region $R_j$.  
However, it is computationally infeasible to consider every possible partition of the feature space. So we take a *top-down, greedy* approach which is called as *recursive binary splitting*.
> *top-down*: The approach starts with the root of the tree   

> *greedy*: We are trying to find the best split at each step rather than looking ahead and picking a split that will lead to a better tree in the future step.   

The *recursive binary splitting* algorithm:
1. Select the predictor $X_j$ and cutpoint $s$ such that splitting the predictor space into the regions $R_1(j, s) = \{X|X_j<s\}$ and $R_2(j, s) = \{X|X_j\ge s\}$ that leads to the greatest RSS reduction. i.e., select the value of $j$ and $s$ that minimizes:
$$ \sum_{i: x_i \in R_1((j, s)} \left( y_i - \hat{y}_{R_1}\right) ^2 + \sum_{i: x_i \in R_2((j, s)} \left( y_i - \hat{y}_{R_2}\right) ^2$$
2. We repeat the process to find the best predictor and best cutpoint that split the above two regions to minimize the RSS within either of the regions. Note that, we will end up split one of the region and the total regions become three.
3. We continue the splitting until no region contains more than $M$ number of observations. Typicall, $M = 5$.

### 2.2 Why and how do we prune the trees?
The *recursive binary splitting* may end up with very good prediction that there is very small RSS (i.e. small bias). In this case, however, the tree may introduce too much complexity to overfit the data and perform very bad on the training data.    

This leads to the process of *tree pruning* which tries to the get best *sub-tree* according to some selected hyper parameter $\alpha$. The methodology is called *cost complexity pruning* or *weakest link pruning*.   

For each value of $\alpha (>0)$ and a full tree $T$, we can find a sub-tree that minimizes the following objective function:
$$\sum_{m = 1}^{|T|}\sum_{i: x_i \in R_m} \left( y_i -\hat{y}_{R_m}\right)^2 + \alpha|T| $$
where $|T|$ is the total number of terminal nodes of tree $T$ and $R_m$ is the area corresponding to the $m^{th}$ terminal node.  

Here is the complete Regression Tree Building Algorithm:
1. Use *recursive binary splitting* to grow a large tree on the trainng data, stopping only when each terminal note has fewer than the minimum number ($M$) of observations.
2. Apply *cost complexity pruning* to the large tree in order to obtain a sequence of best subtress, as a function of $\alpha$.
3. Use *K-fold cross-validation* to choose $\alpha$.
4. Return the subtree from Step 2 that corresponds to the chosen value of $\alpha$.

## 3. Classification Trees
Similar to Regression Trees except that the prediction is for a qualitative value: 
1. Divide the training predictor space (i.e. $X_1, X_2, \cdots, X_p$) into $J$ distinct and non-overlapping regions, $R_1, R_2, \cdots, R_J$.
2. For every observation that falls into the region $R_j$, we use the same value for its prediction. The prediction value is the _most commonly occuring class_ of all the training observation in $R_j$.   

How do we construct the regions then?  
We can still use the _recursive binary splitting_ but obviously we need to change the objective function. And a natural one is the _classification error rate_: the fraction of the training observations in that region that do not belong to the most common class:
$$ \text{Error rate} = 1 - \max_k(\hat{p}_{mk})$$
where $\hat{p}_{mk}$ is the proportion of training observations in the $m$ th region that are from the $k$ th class.  

However, the classification error is not sufficiently sensitive for tree-growing, so in practice, there are two ways to define the objective function:
1. _Gini Index_: It measures the total variance across the $K$ classes.
$$G = \sum_{k=1} ^{K}\hat{p}_{mk} (1 - \hat{p}_{mk}) $$
As can be seen, when $\hat{p}_{mk}$'s are close to 0 or 1, _Gini Index_ takes on small values. When $\hat{p}_{mk}$'s are close to 0 or 1 means that a node contains predominantly observations from a single class. And this is why _Gini Index_ is referred as a measure of _purity_ of a node.

2. _Entropy_: 
$$D = -\sum_{k=1} ^{K}\hat{p}_{mk} \log\hat{p}_{mk}$$
One can show that the entropy will take on a value near 0 if the $\hat{p}_{mk}$’s are all near 0 or near 1. Therefore, like the Gini index, the entropy will take on a small
value if the mth node is pure.

When we are trying to prune the tree, we can use similar procedures as the regression tree and any of the above three approaches might be used. But the classification error rate is preferable if prediction accuracy of the final pruned tree is the goal.

## 4. Advantages and Disadvantage of Decision Trees
* Advantages:
    1. Trees are very easy to explain to people or even a non-expert.
    2. Some people believe that decision trees more closely mirror human decision-making.
    3. Trees can easily handle qualitative predictors without the need to create dummy variables.
* Disadvantages:
    1. Trees generally do not have the same level of predictive accuracy as some of the other regression and classification approaches
    2. Trees can be very non-robust. In other words, a small change in the data can cause a large change in the final estimated tree.

## 5. Ensemble Methods based on Trees

An _ensemble_ method is an approach that combines many simple 'building block' models in order to obtain a single and potentially very powerful model. We will explore the popular _ensemble_ methods: _bagging_, _random forests_ and _boosting_ which can potentially reduce the disadvantages from a single decision tree model.

### 5.1 Bagging
_Boostrap aggregation_ or _bagging_: A general-purpose procedure for reducing the variance of a statistical learning mehtods. The principle is that _averaging a set of observations reduces variance_ since $$\text{var}(\bar{Z}) = \frac{\sigma^2}{n}, Z_i \overset{i.i.d}{\sim} N(\mu, \sigma)$$

In terms of the building block models, we could calculate $\hat{f}^1(x), \cdots, \hat{f}^B(x)$ using $B$ boostrapped training sets, and average them in order to obtain a single low-variance statistical learning model:
$$\hat{f}_{\text{avg}}(x) = \frac{1}{B} \sum_{b=1}^B \hat{f}^b(x)$$

If we combine bagging together with decision tree: growing $B$ unpruned trees and averaging these tree results, it will reduce the variance and improve in accuracy.
Specifically, averaging the tree results means:
* for regression trees, take the average
* for classification tress, take the majority vote (most commonly occuring class among the B predictions).

#### 5.1.1 Out-of-Bag Error Estimation
We can estimate the test error of a bagged model without the need to perform cross-validation or validation set approach.  
One can show that on average, each bagged tree makes use of around two-thirds of the observations. The remaining one-third of the observations not used to fit a given bagged tree are referred to as the _out-of-bag(OOB)_ observations. We can predict each of the OOB observation using the bagged tree and then take the average or majority vote for that observation from all the predictions as the OOB prediction for that observation. This way, we can calculate overall OOB MSE for regression problems or classification error for a classification problem.
#### 5.1.2 Variable Importance Measures
#### 5.1.3 Development of Bagging Methods
* When random subsets of the dataset are drawn as random subsets of the samples, then this algorithm is known as Pasting [B1999].
* When samples are drawn with replacement, then the method is known as Bagging [B1996].
* When random subsets of the dataset are drawn as random subsets of the features, then the method is known as Random Subspaces [H1998].
* When base estimators are built on subsets of both samples and features, then the method is known as Random Patches [LG2012].

### 5.2 Random Forest
_Random Forest_ provide an improvement over bagged trees by way of a small tweak that _decorrelates_ the trees: When building these decision trees, a _random sample of $m$ predictors_ is chosen from as split candidates from the full set of $p$ predictors. We typically choose $m \approx \sqrt{p}$, i.e. not allowed to consider a majority of the available predictors.  

The reason is very simple: if there is a very strong predictor, highly likely each bagged tree will pick this predictor up as the top split. As a result, all the bagged trees will look similar and highly correlate with each other. And this will not reduce a lot of variance. So random forest simply does not allow strong predictors to dominate the bagged tree splits and decorrelates each bagged trees.

So the main difference between _bagging_ and _random forest_ is $m$:
* when $m = p$, it is bagging
* when $m = \sqrt{p}$ or other value that is smaller than $p$

Some notes:
* Using a small value of m in building a random forest will typically be helpful when we have a large number of correlated predictors.
* As with bagging, random forests will not overfit if we increase $B$, so in practice we use a value of $B$ sufficiently large for the error rate to have settled down.

### 5.3 Boosting
Boosting is another way to improve the predictions from a decision tree: Trees are grown _sequentially_: each tree is grown using information from previously grown trees.

The algorithm is as follows:
1. Set $\hat{f}^1(x)$ and $r_i = y_i$ for all $i$ in the training set.
2. For $b = 1, 2, \cdots, B$, repeat:
    * Fit a tree $\hat{f}^b$ with $d$ splits ($d + 1$ terminal nodes) to the training data $(X, r)$
    * Update $\hat{f}$ by adding in a shrunken version of the new tree:
    $$\hat{f}(x) \leftarrow \hat{f}(x) + \lambda \hat{f}^b(x)$$
    * Update the residuals:
    $$r_i \leftarrow r_i - \lambda \hat{f}^b(x)$$
3. Output the boosted model:
$$\hat{f}(x) = \sum_{b = 1}^B \lambda \hat{f}^b(x)$$

There are three tuning parameters:
1. The number of trees $B$. Unlike bagging and random forests, boosting can overfit if B is too large, although this overfitting tends to occur slowly if at all.
2. The shrinkage parameter $\lambda$, a small positive number. This controls the rate at which boosting learns. Typical values are 0.01 or 0.001. Very small $\lambda$ can require using a very large value of B in order to achieve good performance.
3. The number $d$ of splits in each tree, which controls the complexity of the boosted ensemble. Often $d = 1$ works well, in which case each tree is a _stump_, consisting of a single split.

### 5.3.1. Gradient Boosting Machines
The generalization of AdaBoost is Gradient Boosting which allows arbitrary differentiable loss functions to be used. Rather than fit trees to _residuals_, gradient boosting fit trees to _negative gradients_ of any differentiable loss function:

The general steps for both regression and classification are as follows:
* Given Data $\{x_i, y_i\}_{i = 1}^n$ and a differentiable loss function $L$.
* Step 1: Start with an initial model with a constant value: $F_0(x) = \underset{\gamma}{\argmin} \sum_{i = 1} ^n L(y_i, \gamma)$
* Step 2: For $b = 1, 2, \cdots, B$:
    * Calculate negative gradients or pseudo residual: $ r_{ib} = −g(x_i) = −\frac{\partial L(y_i,F(x_i))}{\partial F(x_i)}$ for $i = 1, 2, \cdots, n$.
    * Fit a regression tree to negative gradients $r_{ib} = −g(x_i)$ and create terminal regions $R_{jb}$, for $j = 1, 2, \cdots, J_b$.
    * For $j = 1, 2, \cdots, J_b$: compute $\gamma_{jb} = \underset{\gamma}{\argmin} \sum_{x_i \in R_{ij}} L(y_i, F_{b-1}(x_i) + \gamma)$.
    * $F_b(x) := F_{b-1}(x) + \rho \sum_{j = 1}^{J_b} \gamma_{jb} \cdot I(x \in R_{jb})$, where $\rho$ is the learning rate.

For regressions:
1. We usually use squared loss: $\frac{1}{2}(y_i - \hat{y_i})^2$. It is equivalent to fit to _residuals_ since the negative gradient:
$$−g(x_i) =  −\frac{\partial \frac{1}{2}(y_i - \hat{y_i})^2}{\partial \hat{y_i}} = y_i - \hat{y_i} = \text{residual}$$

For Binary classifications:
1. We usually use negative log-likelihood (log-loss): $-(y_i \times \log \hat{p_i} + (1 - y_i) \times \log (1 - \hat{p_i}))$. And $F(x) = \log \text{odds} (x)$. We can re-write the loss function in terms of log-odds.
$$\begin{aligned}
-(y_i \times \log \hat{p_i} + (1 - y_i) \times \log (1 - \hat{p_i})) & = -y_i \times \log \hat{p_i} - (1 - y_i) \times \log (1 - \hat{p_i}) \\
&= -y_i \times (\log \hat{p_i} - \log (1- \hat{p_i})) - \log (1 - \hat{p_i}) \\
&= -y_i \times \log \text{odds} + \log(1 + \exp ^{\log \text{odds}})
\end{aligned}$$

2. The negative gradient descent w.r.t log odds becomes:

$$
\begin{aligned}
-g(x_i) & = -\frac{\partial}{\partial \log \text{odds}} (-y_i \times \log \text{odds} + \log(1 + \exp ^{\log \text{odds}}))  = -(-y_i + \frac{\exp ^{\log \text{odds}}}{1 + \exp ^{\log \text{odds}}}) = y_i - \hat{p_i}
\end{aligned}
$$

3. How to find $\gamma_{jb} = \underset{\gamma}{\argmin} \sum_{x_i \in R_{ij}} L(y_i, F_{b-1}(x_i) + \gamma)$? To find the optimal value, we will take the first order derivative.
    * First we use Second Order Taylor Expansion at $F_{b-1}(x_i)$ for the Loss function and Then we take the dirivative w.r.t $\gamma$:
$$\begin{aligned} 
L(y_i, F_{b-1}(x_i) + \gamma) & \approx L(y_i, F_{b-1}(x_i)) + \frac{\partial}{\partial F}L(y_i, F_{b-1}(x_i)) \gamma + \frac{1}{2} \frac{\partial^2}{\partial F^2} L(y_i, F_{b-1}(x_i)) \gamma^2 \\
\rightarrow \frac{\partial}{\partial \gamma}L(y_i, F_{b-1}(x_i) +  \gamma) & = \frac{\partial}{\partial F}L(y_i, F_{b-1}(x_i)) + \frac{\partial^2}{\partial F^2} L(y_i, F_{b-1}(x_i)) \gamma \\
\rightarrow \gamma & = \frac{-\frac{\partial}{\partial F}L(y_i, F_{b-1}(x_i))}{\frac{\partial^2}{\partial F^2} L(y_i, F_{b-1}(x_i))} = \frac{y_i - \hat{p_i}}{\hat{p_i} \times (1 - \hat{p_i})} \\
\end{aligned}
$$


Note:
1. Negative gradient pays less attention to outliers than only using residuals and can extend to more differentiable loss functions.


## 7. Classification Examples

In [3]:
# Create Datasets
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=3,
    n_redundant=1,
    n_repeated=0,
    n_classes=2,
    random_state=0,
    shuffle=False,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)


In [ ]:
from sklearn.metrics import roc_auc_score

def eval_print(clf, X_train, y_train, X_test, y_test):
    print('**************************************')
    # Print the training and testing accuracy
    print(f'training accuracy: {clf.score(X_train, y_train)}')
    print(f'testing accuracy: {clf.score(X_test, y_test)}')
    # print training and testing roc_auc_score
    print(f'training AUC score: {roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])}')
    print(f'testing AUC score: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}')

In [18]:
# Bagging classifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

# Just using KNN classifier
knn_clf = KNeighborsClassifier().fit(X_train, y_train)
eval_print(knn_clf, X_train, y_train, X_test, y_test)

# Use KNN classifier as the base estimator
bagging = BaggingClassifier(base_estimator=KNeighborsClassifier(), # The base estimator to fit on random subsets of the dataset. If None, then the base estimator is a DecisionTreeClassifier.
                            n_estimators=100, # The number of base estimators in the ensemble, i.e. number of bagging samples B.
                            max_samples=0.8,  # The number of samples to draw from X to train each base estimator with replacement by default
                            max_features=0.5, # The number of features to draw from X to train each base estimator without replacement by default
                            bootstrap=True, # Whether samples are drawn with replacement.
                            bootstrap_features=False, # Whether features are drawn with replacement.
                            oob_score=True, # Whether to use out-of-bag samples to estimate the generalization error.
                            random_state=23) # Controls both the randomness of the bootstrapping of the samples used when building the base estimator
clf = bagging.fit(X_train, y_train)
eval_print(clf, X_train, y_train, X_test, y_test)
# Print the OOB accuracy score
print(f'training OOB accuracy: {clf.oob_score_}')

**************************************
training accuracy: 0.9333333333333333
testing accuracy: 0.888
training AUC score: 0.9816816475139378
testing AUC score: 0.9482846902201741
**************************************
training accuracy: 0.9613333333333334
testing accuracy: 0.924
training AUC score: 0.9938950676982592
testing AUC score: 0.9643177163338454
training OOB accuracy: 0.912


In [21]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, # The number of trees in the forest.i.e. number of bagging samples B.
                            criterion='gini', # Use 'Gini' as the objective function for the recursive-binary-split
                            max_depth=None, # The maximum depth of the tree
                            min_samples_split=2, # The minimum number of samples required to split an internal node.
                            min_samples_leaf=1, # The minimum number of samples required to be at a leaf node.
                            max_features='sqrt', # the number of features to consider at each split: m = sqrt(p).
                            bootstrap=True, # Whether bootstrap samples are used when building trees.
                            oob_score=True, # Whether to use out-of-bag samples to estimate the generalization score.
                            random_state=23, # Controls both the randomness of the bootstrapping of the samples used when building trees 
                            max_samples=None, # If bootstrap is True, the number of samples to draw from X to train each base estimator. If None (default), then draw X.shape[0] samples.
)
rf_clf = rf.fit(X_train, y_train)
eval_print(rf_clf, X_train, y_train, X_test, y_test)

**************************************
training accuracy: 1.0
testing accuracy: 0.908
training AUC score: 1.0
testing AUC score: 0.9715821812596007


In [29]:
# AdaBoost 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='gini',
                            splitter='best',
                            max_depth=1, # The maximum depth of the tree
                            min_samples_split=2, # The minimum number of samples required to split an internal node.
                            min_samples_leaf=1, # The minimum number of samples required to be at a leaf node.
                            max_features=None, # the number of features to consider at each split: m = sqrt(p).
                            random_state=None
)
adbst = AdaBoostClassifier(base_estimator = dt, # The base estimator from which the boosted ensemble is built. If None, then the base estimator is DecisionTreeClassifier initialized with max_depth=1.
                          n_estimators=50, # The number of trees in the forest.i.e. number of bagging samples B.
                          learning_rate=0.1, # Use 'Gini' as the objective function for the recursive-binary-split
                          random_state = 12, # Controls the random seed given at each base_estimator at each boosting iteration.
)
adbst_clf = rf.fit(X_train, y_train)
eval_print(adbst_clf, X_train, y_train, X_test, y_test)

**************************************
training accuracy: 1.0
testing accuracy: 0.908
training AUC score: 1.0
testing AUC score: 0.9715821812596007


## References
1. Classification and Regression Trees Course Notes
2. Introduction to statistical learning with applications in R
3. https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/
4. http://www.chengli.io/tutorials/gradient_boosting.pdf
5. https://www.youtube.com/@statquest/videos - Gradient Boosting series